# Tensorflow-serving에서 사용할 모델 저장

In [ ]:
# savedmodel로 저장
# tensorflow-serving에서 사용할 모델 저장할 경로
export_path = "./model/BiLSTM/1"
with tf.Graph().as_default():
    sess = tf.Session()
#     tf.Saver로 저장한 모델을 불러올 경로
    save_path = "./model/BiLSTM"
    saver = tf.train.import_meta_graph(save_path+".meta")
    saver.restore(sess, save_path)
    graph = tf.get_default_graph()
    input_X = graph.get_tensor_by_name("BiLSTM/input_X:0")
    dropout_keep_prob = graph.get_tensor_by_name("BiLSTM/dropout_keep_prob:0")
    seq_len = graph.get_tensor_by_name("BiLSTM/seq_len:0")
    output = graph.get_tensor_by_name("BiLSTM/output:0")
#     tensorflow-serving에서 사용할 모델 형식으로 저장
    tf.saved_model.simple_save(sess, export_path, 
                               inputs={
                                   "input_X": input_X,
                                   "dropout_keep_prob": dropout_keep_prob,
                                   "seq_len": seq_len
                               }, 
                               outputs={
                                   "output": output
                               })

# Docker로 tensorflow-serving에 모델 배포

```
docker run -t --rm -p 8501:8501 -v “/E/jhm/open-tube/server/tensorflow/model/BiLSTM:/models/BiLSTM" -e MODEL_NAME=BiLSTM tensorflow/serving
```
* 로컬의 ~model/BiLSTM/1에 모델이 저장된 상태
* 로컬의 ~model/BiLSTM 경로를 Docker 내부의 /models/BiLSTM 경로에 마운트
* Docker 내부의 환경 변수 MODEL_NAME을 모델 이름으로 설정

# Tensorflow-serving에 접근

In [ ]:
from gensim.models import Word2Vec
import tensorflow as tf
import numpy as np
from konlpy.tag import Okt
import sys
import os
import pycurl
import re
from io import BytesIO
import json
import pandas as pd


okt=Okt()
word2vec = Word2Vec.load("./../model/word2vec/word2vec.model")

max_length = 100
input_size = 300

def embedding(word):
    if word in word2vec.wv.vocab:
        return word2vec.wv[word]
    else:
        return np.random.normal(size=input_size)

X = [
    "아 귀여워 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ",
    "아이유 개졸귀 ㅜㅜ너무 귀여워"
]

batch_size = len(X)

# pycurl을 사용해 tensorflow-serving에 전달
c = pycurl.Curl()
c.setopt(c.URL, "http://101.101.164.175:32356/v1/models/BiLSTM:predict")
c.setopt(pycurl.HTTPHEADER, ['Accept: application/json'])

token_X = [["/".join(tag) for tag in okt.pos(sentence, norm=True, stem=True)] for sentence in X]
batch_X = [[embedding(word) for word in sentence] for sentence in token_X]
batch_X_padded = np.zeros(shape=(batch_size, max_length, input_size))
for b in range(batch_size):
    batch_X_padded[b, :len(batch_X[b])] = batch_X[b]
seq_len_ = [len(x) for x in X]

# input을 json 형태로 만듬
data = json.dumps({
    "inputs": {
        "input_X": batch_X_padded.tolist(),
        "dropout_keep_prob": 1.0,
        "seq_len": seq_len_
    }
})

buffer = BytesIO()

c.setopt(c.POST, True)
c.setopt(c.POSTFIELDS, data)
c.setopt(c.WRITEFUNCTION, buffer.write)

c.perform()

# tensorflow serving에 POST로 요청을 보내고 output을 받음
body = buffer.getvalue()
pred = json.loads(body.decode('utf8'))["outputs"]
pred = np.squeeze(pred)
print(pred)

df = pd.DataFrame(pred)

# Kubernetes로 tensorflow serving 배포
## tensorflow_serving.yaml
```
apiVersion: apps/v1
kind: Deployment
metadata:
  name: jhm-tensorflow-serving
  labels:
    service-name: jhm-tensorflow-serving
spec:
  replicas: 1
  selector:
    matchLabels:
      service-name: jhm-tensorflow-serving
  template:
    metadata:
      name: jhm-tensorflow-serving
      labels:
        service-name: jhm-tensorflow-serving
    spec:
      containers:
      - name: jhm-tensorflow-serving
        image: tensorflow/serving
        env:
        - name: MODEL_NAME
          value: "half_plus_two"
        ports:
        - containerPort: 8501
        volumeMounts:
        - name: volumepath
          mountPath: /models/half_plus_two/
      volumes:
      - name: volumepath
        hostPath:
          path: /root/serving/tensorflow_serving/servables/tensorflow/testdata/saved_model_half_plus_two_cpu/
          type: Directory
---
apiVersion: v1
kind: Service
metadata:
  labels:
    run: jhm-tensorflow-serving
  name: jhm-tensorflow-serving
spec:
  ports:
  - port: 8501
    targetPort: 8501
  selector:
    service-name: jhm-tensorflow-serving
  type: NodePort
```
```
kubectl create -f tensorflow.yaml
```

* tensorflow-serving 테스트 모델 배포
* 터미널에서 curl 명령어로 테스트 모델에 접근
```
curl -d '{"instances": [1.0, 2.0, 5.0]}' -X POST http://localhost:8501/v1/models/half_plus_two:predict 
```


In [1]:
import pycurl
from io import BytesIO
import json

c = pycurl.Curl()
c.setopt(c.URL, "http://101.101.167.71:30857/v1/models/half_plus_two:predict")
c.setopt(pycurl.HTTPHEADER, ['Accept: application/json'])

data = json.dumps({
    "instances": [1.0, 2.0, 5.0]
})

buffer = BytesIO()

c.setopt(c.POST, True)
c.setopt(c.POSTFIELDS, data)
c.setopt(c.WRITEFUNCTION, buffer.write)

c.perform()

body = buffer.getvalue()
print(body)

b'{\n    "predictions": [2.5, 3.0, 4.5\n    ]\n}'
